# Library Management System

In [1]:
import sys
import os
import ipynbname

# ============================================
# Initialize Workspace from Main Script
# ============================================
notebook_path = ipynbname.path()
print("Current notebook path:", notebook_path.parent)

# Set up PROJECT_DIR and DATASETS_DIR
PROJECT_DIR = str(notebook_path.parent)
DATASETS_DIR = os.path.join(PROJECT_DIR, "Datasets")

# Create Datasets folder
os.makedirs(DATASETS_DIR, exist_ok=True)
print(f"Project Directory: {PROJECT_DIR}")
print(f"Datasets Directory: {DATASETS_DIR}")

# Set environment variable so modules can use it
os.environ['LMS_DATASETS_DIR'] = DATASETS_DIR

# Add all module directories to path
book_dir = notebook_path.parent / "Book-Management"
loan_dir = notebook_path.parent / "Loan-Management"
member_dir = notebook_path.parent / "Member-Management"

sys.path.insert(0, str(book_dir))
sys.path.insert(0, str(loan_dir))
sys.path.insert(0, str(member_dir))

# Import all modules (they will use the environment variable)
import book
import loan
import member

print("\n" + "="*50)
print("Loading all data...")
print("="*50)

# Load data from all modules
book.load_books()
loan.load_loans()
members_data = member.load_members()

print("\nAll modules loaded successfully!")
print("="*50)

Current notebook path: d:\Project\Github\FajarLaksono\ABW505-Group-77-Assignment\Project-1
Project Directory: d:\Project\Github\FajarLaksono\ABW505-Group-77-Assignment\Project-1
Datasets Directory: d:\Project\Github\FajarLaksono\ABW505-Group-77-Assignment\Project-1\Datasets
Using Datasets directory from main script
Books will be saved to: d:\Project\Github\FajarLaksono\ABW505-Group-77-Assignment\Project-1\Datasets\books.csv
Using Datasets directory from main script
Loans will be saved to: d:\Project\Github\FajarLaksono\ABW505-Group-77-Assignment\Project-1\Datasets\loans.csv
Using Datasets directory from main script
Members will be saved to: d:\Project\Github\FajarLaksono\ABW505-Group-77-Assignment\Project-1\Datasets\members.csv

Loading all data...
File 'd:\Project\Github\FajarLaksono\ABW505-Group-77-Assignment\Project-1\Datasets\books.csv' not found. Starting with empty list.
File 'd:\Project\Github\FajarLaksono\ABW505-Group-77-Assignment\Project-1\Datasets\loans.csv' not found. Start

In [2]:
# Wrapper for borrow_book with validation
def borrow_book_with_validation():
    print("\n=== Borrow Book ===")
    
    # Get and validate book_id
    while True:
        book_id = input("Enter Book ID: ").strip()
        if book_id == "":
            print("Book ID cannot be empty.")
            continue
        if not book.does_book_id_exist(book_id):
            print(f"❌ Book ID '{book_id}' does not exist!")
            retry = input("Try again? (y/n): ").strip().lower()
            if retry != 'y':
                return
            continue
        break
    
    # Get and validate member_id
    while True:
        member_id = input("Enter Member ID: ").strip()
        if member_id == "":
            print("Member ID cannot be empty.")
            continue
        if not member.does_member_id_exist(members_data, member_id):
            print(f"❌ Member ID '{member_id}' does not exist!")
            retry = input("Try again? (y/n): ").strip().lower()
            if retry != 'y':
                return
            continue
        break
    
    # Now call the actual borrow function with pre-filled book_id and member_id
    print("\n✅ Validation passed! Creating loan...")
    
    # Get loan_id
    while True:
        loan_id = input("Enter Loan ID (e.g. L001): ").strip()
        if loan_id == "":
            print("Loan ID cannot be empty.")
        elif loan.is_loan_id_used(loan_id):
            print("This Loan ID already exists.")
        else:
            break
    
    # Get dates
    from datetime import datetime
    while True:
        borrow_date = input("Enter Borrow Date (YYYY-MM-DD): ").strip()
        if borrow_date == "":
            print("Borrow Date cannot be empty.")
        else:
            try:
                datetime.strptime(borrow_date, "%Y-%m-%d")
                break
            except ValueError:
                print("Invalid date format. Please use YYYY-MM-DD.")
    
    while True:
        due_date = input("Enter Due Date (YYYY-MM-DD): ").strip()
        if due_date == "":
            print("Due Date cannot be empty.")
        else:
            try:
                datetime.strptime(due_date, "%Y-%m-%d")
                break
            except ValueError:
                print("Invalid date format. Please use YYYY-MM-DD.")
    
    # Create loan
    new_loan = {
        "loan_id": loan_id,
        "book_id": book_id,
        "member_id": member_id,
        "borrow_date": borrow_date,
        "due_date": due_date,
        "return_date": "",
        "fine": "0"
    }
    
    loan.loans.append(new_loan)
    print(f"\n✅ Loan '{loan_id}' has been created successfully!")

# Wrapper for return_book with validation
def return_book_with_validation():
    print("\n=== Return Book ===")
    
    loan_id = input("Enter Loan ID to return: ").strip()
    
    if loan_id == "":
        print("Loan ID cannot be empty.")
        return
    
    if not loan.is_loan_id_used(loan_id):
        print(f"❌ Loan ID '{loan_id}' does not exist!")
        return
    
    # Call the actual return function
    loan.return_book()

# Wrapper for register member with interactive input
def register_member_interactive():
    print("\n=== Register Member ===")
    
    member_id = input("Enter Member ID (e.g. M001): ").strip()
    name = input("Enter Name: ").strip()
    email = input("Enter Email: ").strip()
    
    if member.register_member(members_data, member_id, name, email):
        print("✅ Member registered successfully!")

In [3]:
# Main Menu Loop
while True:
    print("\n" + "="*50)
    print("     LIBRARY MANAGEMENT SYSTEM")
    print("="*50)
    print("1. Add Book")
    print("2. List Books")
    print("3. Search Books")
    print("4. Register Member")
    print("5. List Members")
    print("6. Borrow Book")
    print("7. Return Book")
    print("8. Save & Exit")
    print("="*50)
    
    choice = input("\nEnter your choice (1-8): ").strip()
    
    if choice == '1':
        book.add_book()
    
    elif choice == '2':
        book.list_books()
    
    elif choice == '3':
        book.search_books()
    
    elif choice == '4':
        register_member_interactive()
    
    elif choice == '5':
        member.list_members(members_data)
    
    elif choice == '6':
        borrow_book_with_validation()
    
    elif choice == '7':
        return_book_with_validation()
    
    elif choice == '8':
        print("\n" + "="*50)
        print("Saving all data...")
        print("="*50)
        
        # Save all modules
        book.save_books()
        loan.save_loans()
        member.save_members(members_data)
        
        print("\n✅ All data saved successfully!")
        print("Thank you for using the Library Management System!")
        print("Goodbye!")
        print("="*50)
        break
    
    else:
        print("\n❌ Invalid choice. Please enter a number between 1 and 8.")


     LIBRARY MANAGEMENT SYSTEM
1. Add Book
2. List Books
3. Search Books
4. Register Member
5. List Members
6. Borrow Book
7. Return Book
8. Save & Exit

=== Add New Book ===

Book '3123' has been added successfully!

     LIBRARY MANAGEMENT SYSTEM
1. Add Book
2. List Books
3. Search Books
4. Register Member
5. List Members
6. Borrow Book
7. Return Book
8. Save & Exit

Saving all data...
Saved 1 book(s) to 'd:\Project\Github\FajarLaksono\ABW505-Group-77-Assignment\Project-1\Datasets\books.csv'.
Saved 0 loan(s) to 'd:\Project\Github\FajarLaksono\ABW505-Group-77-Assignment\Project-1\Datasets\loans.csv'.

✅ All data saved successfully!
Thank you for using the Library Management System!
Goodbye!
